In [1]:
import sys
import os

import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import picaso.justplotit as jpi
import virga.justdoit as vj
import virga.justplotit as cldplt
jpi.output_notebook()
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import glob
import pickle
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
%matplotlib inline
import xarray

Loading BokehJS ...

In [2]:
#1 ck tables from roxana
mh = '+000'#'+0.0' #log metallicity
CtoO = '100'#'1.0' # CtoO ratio

ck_db = f"../data/kcoeff_2020/sonora_2020_feh{mh}_co_{CtoO}.data.196"

#sonora bobcat cloud free structures file
sonora_profile_db = '../data/sonora_bobcat/structures_m+0.0'
sonora_dat_db = '../data/sonora_bobcat/structures_m+0.0'

In [3]:
cl_run = jdi.inputs(calculation="browndwarf", climate = True) # start a calculation

teff= 250 # Effective Temperature of your Brown Dwarf in K
grav = 316 # Gravity of your brown dwarf in m/s/s

cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)')) # input gravity
cl_run.effective_temp(teff) # input effective temperature

opacity_ck = jdi.opannection(ck_db=ck_db) # grab your opacities

In [4]:
nlevel = 91 # number of plane-parallel levels in your code

#Lets set the max and min at 1e-4 bars and 500 bars

# Pmin = 1e-4 #bars
# Pmax = 200 #bars
# pressure=np.logspace(np.log10(Pmin),np.log10(Pmax),nlevel) # set your pressure grid

# temp_guess = np.zeros(shape=(nlevel)) + 500 # K , isothermal atmosphere guess

pressure_bobcat,temp_bobcat = np.loadtxt(jdi.os.path.join(
                            sonora_profile_db,f"t{teff}g{grav}nc_m0.0.dat"),
                            usecols=[1,2],unpack=True, skiprows = 1)

In [5]:
nofczns = 1 # number of convective zones initially

nstr_upper = 60 # top most level of guessed convective zone
nstr_deep = nlevel - 2 # this is always the case. Dont change this
# nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # single convective zone

nstr = np.array([0,nstr_upper,nstr_deep,0,0,0])

# Here are some other parameters needed for the code.
rfacv = 0.0 #we are focused on a brown dwarf so let's keep this as is

In [6]:
cl_run.inputs_climate(temp_guess= temp_bobcat, pressure= pressure_bobcat,
                      nstr = nstr, nofczns = nofczns , rfacv = rfacv, cloudy = "cloudless", mh = '0.0', 
                      CtoO = '1.0',species = ['H2O'], fsed = 8.0, beta = 0.1, virga_param = 'const',
                      mieff_dir = "~/projects/clouds/virga/refrind", do_holes = False, fhole = 0.5, fthin_cld = 0.9, moistgrad = False
                      )

In [7]:
out = cl_run.climate(opacity_ck, save_all_profiles=True,with_spec=True)

Iteration number  0 , min , max temp  79.83677626679349 1322.7016688042108 , flux balance  0.006040328213739502
Iteration number  1 , min , max temp  83.74618078944286 1323.276785445507 , flux balance  0.005994837950146671
Iteration number  2 , min , max temp  87.27330857533819 1323.9593664338665 , flux balance  0.005941468042615865
Iteration number  3 , min , max temp  90.46785025282183 1324.7670777168212 , flux balance  0.005879171269274618
Iteration number  4 , min , max temp  93.37992671584963 1325.7197178153726 , flux balance  0.005806869899423191
Iteration number  5 , min , max temp  96.0493623290581 1326.8360349928375 , flux balance  0.005723701567777737
Iteration number  6 , min , max temp  97.791563069041 1328.1303241494074 , flux balance  0.005629234467041436
Iteration number  7 , min , max temp  98.60258011079016 1329.6097638547246 , flux balance  0.005523577603094917
Iteration number  8 , min , max temp  98.24773684316173 1331.273145759563 , flux balance  0.0054073591333378

In [9]:
opa_mon = jdi.opannection()

hi_res = jdi.inputs(calculation="browndwarf") # start a calculation
teff= 400 # Effective Temperature of your Brown Dwarf in K
grav = 316 # Gravity of your brown dwarf in m/s/s
hi_res.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)')) # input gravity

hi_res.atmosphere(df=out['ptchem_df'])
# hi_res.clouds(df=out['cld_output_picaso'])

df_spec1 = hi_res.spectrum(opa_mon, calculation='thermal', full_output= True)

wno, fp = df_spec1['wavenumber'], df_spec1['thermal'] #erg/cm2/s/cm
xmicron = 1e4/wno

flamy = fp*1e-8  # per anstrom instead of per cm
sp = jdi.psyn.ArraySpectrum(xmicron, flamy,
                           waveunits='um',
                           fluxunits='FLAM')
sp.convert("um")
sp.convert('Fnu')  # erg/cm2/s/Hz

wno = sp.wave  # micron
fp = sp.flux  # erg/cm2/s/Hz
df_spec1['fluxnu'] = fp
wno,fp = jdi.mean_regrid(1e4/wno,fp, R=200)

In [10]:
plt.loglog(1e4/wno,fp)
plt.show()